In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
df = pd.read_csv('train.csv').drop(columns=['Id'])
X = np.array(df.iloc[:, 1:])
y = to_categorical(np.array(df.iloc[:, 0]))

# Convert the training and test images into 3 channels
X = np.dstack([X] * 3)
# Reshape images as per the tensor format required by tensorflow
X = X.reshape(-1, 28,28,3)
X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((150,150))) for im in X])
X = X.astype('float32')/255

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(train_features, y_train, batch_size=128, epochs=50, verbose=1, validation_data=(val_features, y_val))

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

model.fit(train_features, y_train, batch_size=128, epochs=50, verbose=1, validation_data=(val_features, y_val))